In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [ ]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [2]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
model = MultVAERecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])
earlystopping_kwargs =  {"validation_every_n": 10,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": "MAP",
                              }

2023-11-29 12:27:12.928378: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 12:27:13.498340: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 12:27:13.501463: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 12:27:14.662743: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


EvaluatorHoldout: Ignoring 2664 (21.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op
def objective(trial):
    n_users, n_items = URM_train.shape
    epochs = trial.suggest_int("epochs", 500, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)
    l2_reg = trial.suggest_float("l2_reg", 1e-6, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0., 0.8)
    total_anneal_steps = trial.suggest_int("total_anneal_steps", 100000, 600000)
    anneal_cap = trial.suggest_float("anneal_cap", 0., 0.6)
    batch_size = trial.suggest_categorical("batch_size", [128, 256, 512, 1024])
    encoding_size = trial.suggest_int("encoding_size", 1, min(512, n_items-1))
    next_layer_size_multiplier = trial.suggest_int("next_layer_size_multiplier", 2, 10)
    max_n_hidden_layers = trial.suggest_int("max_n_hidden_layers", 1, 4)
    max_parameters = trial.suggest_categorical("max_parameters", [7*1e9*8/32])
    
    recommender = model(URM_train, force_gpu=True)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [4]:
best_params = {
    "epochs": 500,
    "learning_rate": 0.0001,
    "l2_reg": 0.0001,
    "dropout": 0.2,
    "total_anneal_steps": 200000,
    "anneal_cap": 0.2,
    "batch_size": 512,
    "encoding_size": 128,
    "next_layer_size_multiplier": 2,
    "max_n_hidden_layers": 2,
    "max_parameters": 7*1e9*8/32
}

study_name = "multvae-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize")
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=30)

[I 2023-11-29 12:30:54,361] A new study created in RDB with name: multvae-study


MultVAERecommender: URM Detected 443 ( 3.5%) users with no interactions.
MultVAERecommender: URM Detected 316 ( 1.4%) items with no interactions.


2023-11-29 12:30:54.580228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-29 12:30:54.921953: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
[W 2023-11-29 12:30:54,939] Trial 0 failed with parameters: {'epochs': 500, 'learning_rate': 0.0001, 'l2_reg': 0.0001, 'dropout': 0.2, 'total_anneal_steps': 200000, 'anneal_cap': 0.2, 'batch_size': 512, 'encoding_size': 128, 'next_layer_size_multiplier': 2, 'max_n_h

AssertionError: 

In [9]:
study.best_params

{'topK': 101, 'alpha': 0.3026342852596128, 'beta': 0.058468783118329024}

In [10]:
final = model(URM_train_validation)
final.fit(**study.best_params)

RP3betaRecommender: URM Detected 245 ( 1.9%) users with no interactions.
RP3betaRecommender: URM Detected 110 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4027.41 column/sec. Elapsed time 5.52 sec


In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2100 (16.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10538 (100.0%) in 5.18 sec. Users per second: 2033


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.090378                 0.157221  0.133194  0.045665    0.078973   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.259414  0.143182  0.107686  0.509869      0.339649  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.833834          0.425146    0.833834       0.057877   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.145296                   0.997617             0.165954   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   